In [ ]:
from transformers import BertTokenizer
from modulos.utils import  labeler
from modulos.model import split_data, token_data, create_model, performance
import pandas as pd
import modulos.limpieza as clean_data
import modulos.load_data as ldata

In [ ]:
from tensorflow.keras import layers, models
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, f1_score, classification_report

def split_data(df):
    from sklearn.model_selection import train_test_split
    x = df['text'] 
    y = df['label'] 
    return train_test_split(x,y,test_size = 0.2,random_state=42)

def token_data(texts, tokenizer, max_length=512):
    encodings = tokenizer(
        texts,
        truncation=True,
        padding=True,
        max_length=max_length,
        return_tensors='tf' 
    )
    return encodings

def performance(y_true,y_pred):
# y_pred son los valores predecidos mientras que y_true son los valores reales
    accuracy = accuracy_score(y_true,y_pred)
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    print(classification_report(y_true, y_pred))
    return{
        'accuracy':accuracy,
        'recall':recall,
        'f1':f1,
    }


def create_model(inputs, num,  num_filters=128, kernel_size=5, pool_size=2):
    model = models.Sequential()
    model.add(layers.Conv1D(num_filters, kernel_size, activation='gelu', input_shape=inputs))
    model.add(layers.MaxPooling1D(pool_size=pool_size))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv1D(num_filters * 2, kernel_size, activation='gelu'))
    model.add(layers.MaxPooling1D(pool_size=pool_size))
    model.add(layers.BatchNormalization())
    model.add(layers.GlobalAveragePooling1D())
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(128, activation = 'gelu'))
    model.add(layers.Dense(num,activation = 'softmax'))
    return model

In [ ]:
def categorize(text):
    categories = {
        "malware": "Malware",
        "phishing": "Phishing",
        "ransomware": "Ransomware",
        "trojan": "Trojan",
        "adware": "Adware",
        "worm": "Worm",
        "spyware": "Spyware",
        "ddos": "DDoS",
        "distributed denial of service": "DDoS",
        "zero day": "Zero Days",
        "data breach": "Data Breach",
        "social engineering": "Social Engineering"
    }
    lower_text = text.lower()
    for keyword, category in categories.items():
        if keyword in lower_text:
            return category
    return "Other" 

def labeler(df):
    df["label"] = df["text"].apply(categorize)
    return df

In [ ]:
def main():
    # LOAD
    #df = clean_data.limpieza(ldata.ldata1(), ldata.ldata2(), ldata.ldata3())
    df = pd.read_csv('./datasets/clean_data.csv')
    print("data loaded")
    #LABEL
    df = labeler(df) 

    # SPLIT
    x_train, x_test, y_train, y_test = split_data(df)

    # TOKENIZE
    train_texts = x_train.tolist()  
    val_texts = x_test.tolist()  
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    train_encodings = token_data(train_texts, tokenizer)
    val_encodings = token_data(val_texts,tokenizer)
    
    # Actually training the model
    y_train = pd.get_dummies(y_train).values
    y_test = pd.get_dummies(y_test).values
    inputs = (train_encodings['input_ids'].shape[1], 1) 
    num = y_train.shape[1]
    print("creando modelo")
    model = create_model(inputs, num)

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(train_encodings['input_ids'],y_train,epochs=5, batch_size=32, validation_data=(val_encodings['input_ids'],y_test))
#    y_pred =[] # reemplazar
#    y_true = [] # reemplazar 
#    metrics = performance(y_true, y_pred)
#    print(metrics)


if __name__ == "__main__":
    main()